## 1. Place Human, Furnitures in the Mujoco Simulator

In [1]:
import sys,mujoco,time,os,json
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../package/helper/')
sys.path.append('../package/mujoco_usage/')
sys.path.append('../package/gpt_usage/')
sys.path.append('../package/detection_module/')
from mujoco_parser import *
from utility import *
from transformation import *
from gpt_helper import *
from owlv2 import *
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("Ready.")

Ready.


In [2]:
# Initialize environment
xml_path = '../asset/makeup_frida/scene_table.xml'
env = MuJoCoParserClass(name='MakeupFrida', rel_xml_path=xml_path, verbose=True)
print("Done.")

# Initialize viewer
env.reset()
env.init_viewer(
    transparent=False,
    azimuth=105,
    distance=3.12,
    elevation=-29,
    lookat=[0.39, 0.25, 0.43],
)

# Get object names
obj_names = env.get_body_names(prefix='obj_')
n_obj = len(obj_names)
print("Object list:")
for obj_idx in range(n_obj):
    print(f" [{obj_idx}] obj_name:[{obj_names[obj_idx]}]")

# Main loop
env_state = env.get_state()
while env.is_viewer_alive():
    # Step
    env.step()
    
    # Render
    if env.loop_every(tick_every=10):
        env.plot_time()
        env.render()
    
    # Plot
    if env.loop_every(tick_every=500):  # every 1 second
        # Grab current view
        render_img = env.grab_image(rsz_rate=1/4)
        # Plot
        plt.figure(figsize=(8, 3))
        plt.imshow(render_img)
        plt.title(f"Time:[{env.get_sim_time():.2f}]sec", fontsize=10)
        plt.show()
        

env.close_viewer()
print ("Done.")


name:[MakeupFrida] dt:[0.002] HZ:[500]
n_qpos:[19] n_qvel:[18] n_qacc:[18] n_ctrl:[7]

n_body:[23]
 [0/23] [world] mass:[0.00]kg
 [1/23] [base_table] mass:[100.00]kg
 [2/23] [object_table] mass:[100.00]kg
 [3/23] [ur_base] mass:[4.00]kg
 [4/23] [ur_shoulder_link] mass:[3.70]kg
 [5/23] [ur_upper_arm_link] mass:[8.39]kg
 [6/23] [ur_forearm_link] mass:[2.27]kg
 [7/23] [ur_wrist_1_link] mass:[1.22]kg
 [8/23] [ur_wrist_2_link] mass:[1.22]kg
 [9/23] [ur_wrist_3_link] mass:[0.19]kg
 [10/23] [ur_tcp_link] mass:[0.00]kg
 [11/23] [ur_camera_mount] mass:[0.09]kg
 [12/23] [ur_d435i] mass:[0.07]kg
 [13/23] [ur_rg2_gripper_base_link] mass:[0.20]kg
 [14/23] [ur_camera_center] mass:[0.00]kg
 [15/23] [ur_rg2_gripper_finger1_finger_link] mass:[0.01]kg
 [16/23] [ur_rg2_gripper_finger1_inner_knuckle_link] mass:[0.01]kg
 [17/23] [ur_rg2_gripper_finger1_finger_tip_link] mass:[0.01]kg
 [18/23] [ur_rg2_gripper_finger2_finger_link] mass:[0.01]kg
 [19/23] [ur_rg2_gripper_finger2_inner_knuckle_link] mass:[0.01]k

: 